In [ ]:
!pip install kaggle

In [1]:
from google.colab import files

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Download the Dataset
!kaggle datasets download -d arashnic/book-recommendation-dataset

# Unzip the Dataset
import zipfile
import os

# Create a directory to unzip the dataset
os.makedirs('book-recommendation-dataset', exist_ok=True)
# Unzip the dataset
with zipfile.ZipFile('/content/book-recommendation-dataset.zip', 'r') as zip_ref:
    zip_ref.extractall('book-recommendation-dataset')

print('Dataset downloaded and extracted successfully.')

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Dataset URL: https://www.kaggle.com/datasets/arashnic/book-recommendation-dataset
License(s): CC0-1.0
 99% 24.0M/24.3M [00:01<00:00, 27.2MB/s]
100% 24.3M/24.3M [00:01<00:00, 16.2MB/s]
Dataset downloaded and extracted successfully.


In [39]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
book_df = pd.read_csv('/content/book-recommendation-dataset/Books.csv')
user_df = pd.read_csv('/content/book-recommendation-dataset/Users.csv')
rating_df = pd.read_csv('/content/book-recommendation-dataset/Ratings.csv')

In [4]:
book_df

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...
...,...,...,...,...,...,...,...,...
271355,0440400988,There's a Bat in Bunk Five,Paula Danziger,1988,Random House Childrens Pub (Mm),http://images.amazon.com/images/P/0440400988.0...,http://images.amazon.com/images/P/0440400988.0...,http://images.amazon.com/images/P/0440400988.0...
271356,0525447644,From One to One Hundred,Teri Sloat,1991,Dutton Books,http://images.amazon.com/images/P/0525447644.0...,http://images.amazon.com/images/P/0525447644.0...,http://images.amazon.com/images/P/0525447644.0...
271357,006008667X,Lily Dale : The True Story of the Town that Ta...,Christine Wicker,2004,HarperSanFrancisco,http://images.amazon.com/images/P/006008667X.0...,http://images.amazon.com/images/P/006008667X.0...,http://images.amazon.com/images/P/006008667X.0...
271358,0192126040,Republic (World's Classics),Plato,1996,Oxford University Press,http://images.amazon.com/images/P/0192126040.0...,http://images.amazon.com/images/P/0192126040.0...,http://images.amazon.com/images/P/0192126040.0...


In [5]:
user_df

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN
...,...,...,...
278853,278854,"portland, oregon, usa",NaN
278854,278855,"tacoma, washington, united kingdom",50.0
278855,278856,"brampton, ontario, canada",NaN
278856,278857,"knoxville, tennessee, usa",NaN


In [6]:
rating_df

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6
...,...,...,...
1149775,276704,1563526298,9
1149776,276706,0679447156,0
1149777,276709,0515107662,10
1149778,276721,0590442449,10


In [8]:
book_df.isnull().sum()

ISBN                   0
Book-Title             0
Book-Author            2
Year-Of-Publication    0
Publisher              2
Image-URL-S            0
Image-URL-M            0
Image-URL-L            3
dtype: int64

In [10]:
book_df.dropna(inplace=True)

In [12]:
user_df.isnull().sum()

User-ID          0
Location         0
Age         110762
dtype: int64

In [13]:
rating_df.isnull().sum()

User-ID        0
ISBN           0
Book-Rating    0
dtype: int64

In [16]:
book_df.duplicated().sum()

0

In [17]:
user_df.duplicated().sum()

0

In [19]:
rating_df.duplicated().sum()

0

In [77]:
df = book_df.merge(rating_df, on='ISBN', how='inner')

In [21]:
df

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L,User-ID,Book-Rating
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,2,0
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,8,5
2,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,11400,0
3,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,11676,8
4,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,41385,0
...,...,...,...,...,...,...,...,...,...,...
1031123,0440400988,There's a Bat in Bunk Five,Paula Danziger,1988,Random House Childrens Pub (Mm),http://images.amazon.com/images/P/0440400988.0...,http://images.amazon.com/images/P/0440400988.0...,http://images.amazon.com/images/P/0440400988.0...,276463,7
1031124,0525447644,From One to One Hundred,Teri Sloat,1991,Dutton Books,http://images.amazon.com/images/P/0525447644.0...,http://images.amazon.com/images/P/0525447644.0...,http://images.amazon.com/images/P/0525447644.0...,276579,4
1031125,006008667X,Lily Dale : The True Story of the Town that Ta...,Christine Wicker,2004,HarperSanFrancisco,http://images.amazon.com/images/P/006008667X.0...,http://images.amazon.com/images/P/006008667X.0...,http://images.amazon.com/images/P/006008667X.0...,276680,0
1031126,0192126040,Republic (World's Classics),Plato,1996,Oxford University Press,http://images.amazon.com/images/P/0192126040.0...,http://images.amazon.com/images/P/0192126040.0...,http://images.amazon.com/images/P/0192126040.0...,276680,0


In [22]:
df.isnull().sum()

ISBN                   0
Book-Title             0
Book-Author            0
Year-Of-Publication    0
Publisher              0
Image-URL-S            0
Image-URL-M            0
Image-URL-L            0
User-ID                0
Book-Rating            0
dtype: int64

### Collaborative filtering based book recommendation system

In [30]:
# defining some criteria of valid user who rated minimum of 250 books
# and famous book which is rated by minimum of 50 users

min_ratings_threshold = 200
famous_books_threshold = 100

In [31]:
num_ratings_per_user = df.groupby('User-ID')['Book-Rating'].count()
valid_users = num_ratings_per_user[num_ratings_per_user > min_ratings_threshold].index

In [32]:
num_ratings_per_book = df.groupby('Book-Title')['Book-Rating'].count()
famous_books = num_ratings_per_book[num_ratings_per_book > famous_books_threshold].index

In [33]:
filtered_df = df[df['User-ID'].isin(valid_users)]

In [34]:
final_df = filtered_df[filtered_df['Book-Title'].isin(famous_books)]

In [35]:
final_df

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L,User-ID,Book-Rating
31,0399135782,The Kitchen God's Wife,Amy Tan,1991,Putnam Pub Group,http://images.amazon.com/images/P/0399135782.0...,http://images.amazon.com/images/P/0399135782.0...,http://images.amazon.com/images/P/0399135782.0...,11676,9
33,0399135782,The Kitchen God's Wife,Amy Tan,1991,Putnam Pub Group,http://images.amazon.com/images/P/0399135782.0...,http://images.amazon.com/images/P/0399135782.0...,http://images.amazon.com/images/P/0399135782.0...,36836,0
34,0399135782,The Kitchen God's Wife,Amy Tan,1991,Putnam Pub Group,http://images.amazon.com/images/P/0399135782.0...,http://images.amazon.com/images/P/0399135782.0...,http://images.amazon.com/images/P/0399135782.0...,46398,9
38,0399135782,The Kitchen God's Wife,Amy Tan,1991,Putnam Pub Group,http://images.amazon.com/images/P/0399135782.0...,http://images.amazon.com/images/P/0399135782.0...,http://images.amazon.com/images/P/0399135782.0...,113270,0
39,0399135782,The Kitchen God's Wife,Amy Tan,1991,Putnam Pub Group,http://images.amazon.com/images/P/0399135782.0...,http://images.amazon.com/images/P/0399135782.0...,http://images.amazon.com/images/P/0399135782.0...,113519,0
...,...,...,...,...,...,...,...,...,...,...
1027946,0613032950,Animal Dreams,Barbara Kingsolver,1990,Sagebrush Bound,http://images.amazon.com/images/P/0613032950.0...,http://images.amazon.com/images/P/0613032950.0...,http://images.amazon.com/images/P/0613032950.0...,238120,0
1028406,1878702831,Echoes,Nancy Morse,1992,Meteor Publishing Corporation,http://images.amazon.com/images/P/1878702831.0...,http://images.amazon.com/images/P/1878702831.0...,http://images.amazon.com/images/P/1878702831.0...,238781,0
1028592,0394429869,I Know Why the Caged Bird Sings,Maya Angelou,1996,Random House,http://images.amazon.com/images/P/0394429869.0...,http://images.amazon.com/images/P/0394429869.0...,http://images.amazon.com/images/P/0394429869.0...,239594,8
1028807,0743527631,The Pillars of the Earth,Ken Follett,2002,Encore,http://images.amazon.com/images/P/0743527631.0...,http://images.amazon.com/images/P/0743527631.0...,http://images.amazon.com/images/P/0743527631.0...,240144,0


In [36]:
book_pt = final_df.pivot_table(index='Book-Title', columns='User-ID', values='Book-Rating')

In [37]:
book_pt

User-ID,254,2276,2766,2977,3363,4017,4385,6251,6323,6543,...,271705,273979,274004,274061,274301,274308,275970,277427,277639,278418
Book-Title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,10.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN
1st to Die: A Novel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24 Hours,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,10.0,NaN,NaN
2nd Chance,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN
4 Blondes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Year of Wonders,NaN,NaN,NaN,7.0,NaN,NaN,NaN,NaN,NaN,0.0,...,NaN,9.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN
You Belong To Me,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN


In [38]:
# filling null values with 0
book_pt.fillna(0,inplace=True)
book_pt

User-ID,254,2276,2766,2977,3363,4017,4385,6251,6323,6543,...,271705,273979,274004,274061,274301,274308,275970,277427,277639,278418
Book-Title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
24 Hours,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0
2nd Chance,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Year of Wonders,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
You Belong To Me,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [42]:
similarity_score = cosine_similarity(book_pt)
similarity_score.shape

(904, 904)

In [43]:
# now declaring a function which take input a book and recommend books based on similarity scores

In [47]:
def recommend_books(book_name):
  book_index = book_pt.index.get_loc(book_name)
  similar_books = list(enumerate(similarity_score[book_index]))
  similar_books = sorted(similar_books, key=lambda x:x[1], reverse=True)[1:7]

  for i in similar_books:
    print(book_pt.index[i[0]])
    books = []

    for i in similar_books:
        data = []
        recommended_book = book_df[book_df['Book-Title'] == book_pt.index[i[0]]]
        data.extend(list(recommended_book.drop_duplicates('Book-Title')['Book-Title'].values))
        data.extend(list(recommended_book.drop_duplicates('Book-Title')['Book-Author'].values))
        data.extend(list(recommended_book.drop_duplicates('Book-Title')['Image-URL-M'].values))

        books.append(data)
    return books

In [48]:
recommend_books("You Belong To Me")

Loves Music, Loves to Dance


[['Loves Music, Loves to Dance',
  'Mary Higgins Clark',
  'http://images.amazon.com/images/P/0671758896.01.MZZZZZZZ.jpg'],
 ["I'll Be Seeing You",
  'Mary Higgins Clark',
  'http://images.amazon.com/images/P/0671888587.01.MZZZZZZZ.jpg'],
 ['Before I Say Good-Bye',
  'Mary Higgins Clark',
  'http://images.amazon.com/images/P/0671004573.01.MZZZZZZZ.jpg'],
 ["Daddy's Little Girl",
  'Mary Higgins Clark',
  'http://images.amazon.com/images/P/0743206045.01.MZZZZZZZ.jpg'],
 ['All Around the Town',
  'Mary Higgins Clark',
  'http://images.amazon.com/images/P/0671793489.01.MZZZZZZZ.jpg'],
 ['While My Pretty One Sleeps',
  'Mary Higgins Clark',
  'http://images.amazon.com/images/P/0671673688.01.MZZZZZZZ.jpg']]

In [49]:
recommend_books("The Handmaid's Tale")

Bastard Out of Carolina


[['Bastard Out of Carolina',
  'Dorothy Allison',
  'http://images.amazon.com/images/P/0452269571.01.MZZZZZZZ.jpg'],
 ['A Civil Action',
  'JONATHAN HARR',
  'http://images.amazon.com/images/P/0679772677.01.MZZZZZZZ.jpg'],
 ['1984',
  'George Orwell',
  'http://images.amazon.com/images/P/0451524934.01.MZZZZZZZ.jpg'],
 ['The Bone Collector (Lincoln Rhyme Novels (Paperback))',
  'Jeff Deaver',
  'http://images.amazon.com/images/P/0451188454.01.MZZZZZZZ.jpg'],
 ["The Kitchen God's Wife",
  'Amy Tan',
  'http://images.amazon.com/images/P/0399135782.01.MZZZZZZZ.jpg'],
 ['The Red Tent (Bestselling Backlist)',
  'Anita Diamant',
  'http://images.amazon.com/images/P/0312195516.01.MZZZZZZZ.jpg']]

In [51]:
recommend_books("Animal Dreams")

The Bean Trees


[['The Bean Trees',
  'Barbara Kingsolver',
  'http://images.amazon.com/images/P/0060915544.01.MZZZZZZZ.jpg'],
 ['Daughter of Fortune',
  'Isabel Allende',
  'http://images.amazon.com/images/P/0060932759.01.MZZZZZZZ.jpg'],
 ['Ladder of Years',
  'Anne Tyler',
  'http://images.amazon.com/images/P/0679441557.01.MZZZZZZZ.jpg'],
 ['Atonement: A Novel',
  'Ian McEwan',
  'http://images.amazon.com/images/P/0385503954.01.MZZZZZZZ.jpg'],
 ['Cold Mountain : A Novel',
  'CHARLES FRAZIER',
  'http://images.amazon.com/images/P/0375700757.01.MZZZZZZZ.jpg'],
 ['Winter Solstice',
  'Rosamunde Pilcher',
  'http://images.amazon.com/images/P/0312244266.01.MZZZZZZZ.jpg']]

In [52]:
recommend_books("Harry Potter and the Prisoner of Azkaban (Book 3)")

Harry Potter and the Goblet of Fire (Book 4)


[['Harry Potter and the Goblet of Fire (Book 4)',
  'J. K. Rowling',
  'http://images.amazon.com/images/P/0439139597.01.MZZZZZZZ.jpg'],
 ['Harry Potter and the Chamber of Secrets (Book 2)',
  'J. K. Rowling',
  'http://images.amazon.com/images/P/0439064872.01.MZZZZZZZ.jpg'],
 ['Harry Potter and the Order of the Phoenix (Book 5)',
  'J. K. Rowling',
  'http://images.amazon.com/images/P/043935806X.01.MZZZZZZZ.jpg'],
 ["Harry Potter and the Sorcerer's Stone (Book 1)",
  'J. K. Rowling',
  'http://images.amazon.com/images/P/0590353403.01.MZZZZZZZ.jpg'],
 ["Harry Potter and the Sorcerer's Stone (Harry Potter (Paperback))",
  'J. K. Rowling',
  'http://images.amazon.com/images/P/059035342X.01.MZZZZZZZ.jpg'],
 ['The Fellowship of the Ring (The Lord of the Rings, Part 1)',
  'J.R.R. TOLKIEN',
  'http://images.amazon.com/images/P/0345339703.01.MZZZZZZZ.jpg']]

In [100]:
# saving in pickle file
import pickle
pickle.dump(similarity_score,open('similarity_score.pkl','wb'))
pickle.dump(book_pt,open('book_pt.pkl','wb'))
pickle.dump(book_df,open('book_df.pkl','wb'))

### Popularity Based Recommendation System

In [53]:
# We will recommend top 100 books based on popularity

In [78]:
popular_books = df.groupby('ISBN').agg({
        'Book-Rating': ['count', 'mean'],
        'Book-Title': 'first',
        'Book-Author': 'first',
        'Image-URL-M': 'first'
    }).reset_index()

In [79]:
popular_books.head()

ISBN Book-Rating       \
                    count mean   
0  0000913154           1  8.0   
1  0001010565           2  0.0   
2  0001046438           1  9.0   
3  0001046713           1  0.0   
4  000104687X           1  6.0   

                                          Book-Title  \
                                               first   
0  The Way Things Work: An Illustrated Encycloped...   
1                                    Mog's Christmas   
2                                               Liar   
3                       Twopence to Cross the Mersey   
4  T.S. Eliot Reading \The Wasteland\" and Other ...   

                     Book-Author  \
                           first   
0  C. van Amerongen (translator)   
1                    Judith Kerr   
2                    Stephen Fry   
3                Helen Forrester   
4                     T.S. Eliot   

                                         Image-URL-M  
                                               first  
0  http://images.amazon.com/images/P/0000913154.0...  
1  http://images.amazon.com/images/P/0001010565.0...  
2  http://images.amazon.com/images/P/0001046438.0...  
3  http://images.amazon.com/images/P/0001046713.0...  
4  http://images.amazon.com/images/P/000104687X.0...

In [80]:
popular_books.columns = ['ISBN', 'num_ratings', 'avg_rating', 'Title', 'Author', 'Image_URL']
popular_books.head(2)

,ISBN,num_ratings,avg_rating,Title,Author,Image_URL
0,0000913154,1,8.0,The Way Things Work: An Illustrated Encycloped...,C. van Amerongen (translator),http://images.amazon.com/images/P/0000913154.0...
1,0001010565,2,0.0,Mog's Christmas,Judith Kerr,http://images.amazon.com/images/P/0001010565.0...


In [91]:
# for popularity defining min threshold for famous book
famous_book_threshold = 230

In [96]:
top_books = popular_books[popular_books['num_ratings']>=famous_book_threshold].sort_values('avg_rating', ascending=False).head(100)

In [97]:
top_books=top_books.drop_duplicates('ISBN')

In [98]:
top_books

,ISBN,num_ratings,avg_rating,Title,Author,Image_URL
84467,043935806X,334,5.571856,Harry Potter and the Order of the Phoenix (Boo...,J. K. Rowling,http://images.amazon.com/images/P/043935806X.0...
43238,0345339681,281,5.007117,The Hobbit : The Enchanting Prelude to The Lor...,J.R.R. TOLKIEN,http://images.amazon.com/images/P/0345339681.0...
91208,0446310786,389,4.920308,To Kill a Mockingbird,Harper Lee,http://images.amazon.com/images/P/0446310786.0...
125012,059035342X,571,4.900175,Harry Potter and the Sorcerer's Stone (Harry P...,J. K. Rowling,http://images.amazon.com/images/P/059035342X.0...
83626,0439064872,351,4.729345,Harry Potter and the Chamber of Secrets (Book 2),J. K. Rowling,http://images.amazon.com/images/P/0439064872.0...
...,...,...,...,...,...,...
174367,0804106304,519,3.063584,The Joy Luck Club,Amy Tan,http://images.amazon.com/images/P/0804106304.0...
37202,0312983271,257,3.042802,Full House (Janet Evanovich's Full Series),Janet Evanovich,http://images.amazon.com/images/P/0312983271.0...
62134,0375727345,552,3.039855,House of Sand and Fog,Andre Dubus III,http://images.amazon.com/images/P/0375727345.0...
121285,0553572997,309,3.016181,The Alienist,Caleb Carr,http://images.amazon.com/images/P/0553572997.0...


In [99]:
# saving in pickle file
import pickle
pickle.dump(top_books,open('top_books.pkl','wb'))